# 事前準備

## Googleドライブのマウント

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 地域コード一覧を作成

兵庫県内の全市区町村

In [94]:
import json

def areas(prefCode):
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    areas = list(filter(lambda x: x['prefCode'] == prefCode, arealist))
    return areas

In [95]:
areas = areas(28)
areas

[{'index': 1296,
  'prefCode': 28,
  'prefName': '兵庫県',
  'cityCode': '28100',
  'cityName': '神戸市',
  'bigCityFlag': '2',
  'areaCode': '28100',
  'areaName': '神戸市',
  'governmentType': 'city'},
 {'index': 1297,
  'prefCode': 28,
  'prefName': '兵庫県',
  'cityCode': '28101',
  'cityName': '神戸市東灘区',
  'bigCityFlag': '1',
  'areaCode': '28101',
  'areaName': '神戸市東灘区',
  'governmentType': 'city'},
 {'index': 1298,
  'prefCode': 28,
  'prefName': '兵庫県',
  'cityCode': '28102',
  'cityName': '神戸市灘区',
  'bigCityFlag': '1',
  'areaCode': '28102',
  'areaName': '神戸市灘区',
  'governmentType': 'city'},
 {'index': 1299,
  'prefCode': 28,
  'prefName': '兵庫県',
  'cityCode': '28105',
  'cityName': '神戸市兵庫区',
  'bigCityFlag': '1',
  'areaCode': '28105',
  'areaName': '神戸市兵庫区',
  'governmentType': 'city'},
 {'index': 1300,
  'prefCode': 28,
  'prefName': '兵庫県',
  'cityCode': '28106',
  'cityName': '神戸市長田区',
  'bigCityFlag': '1',
  'areaCode': '28106',
  'areaName': '神戸市長田区',
  'governmentType': 'city'},
 {'

# estat-APIからデータを取得する関数

In [92]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

import urllib.parse
import urllib.request
import json
import pandas as pd
import numpy as np

"""
estat-APIのレスポンスを整形してDataFrameに変換
"""
def get_estat_dataframe(params):
  res = json_response(params)
  df_res = conv_estat_response_to_dataframe(res)

  return df_res

"""
estat-APIのレスポンスを取得する関数
"""
def json_response(params):
  p = params.copy()
  p['appId'] = '724e5b90772a3e9289f41a253e4e7e32438f4fff'
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))


"""
estat-APIのレスポンスをDataFrameに変換する
"""
def conv_estat_response_to_dataframe(response):

  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # times
  dict_times = list(filter(lambda item : item['@id'] == 'time', CLASS_OBJ))[0]['CLASS']
  
  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']
  df = pd.json_normalize(VALUE).replace('-', '0').replace('X', '0').astype({'$': 'int'})

  # 年次を列展開
  for time in dict_times:
    df[time['@name']] = df.apply(lambda x: x['$'] if x['@time'] == time['@code'] else 0, 1)

  # 集計用の列dictを作成
  columns = {t: 'sum' for t in [d.get('@name') for d in dict_times]}
  
  # 集計
  df = df.groupby(['@area','@unit']).agg(columns).reset_index()
  df = df.replace(0, np.nan).dropna(how='all', axis=1)
  print(df.head(1).iat[0, 2])
  # df_c = df_c.astype({categoryName: int}) if float(df_c.head(1).iat[0, 1]).is_integer() else df_c.astype({categoryName: float})
  return df

In [24]:
params = {
    'statsDataId':'0000020201',
    'cdCat01':'A1101',
    }

In [91]:
df_population_city = get_estat_dataframe(params)
df_population_city

人


,@area,@unit,1980年度,1985年度,1990年度,1995年度,2000年度,2005年度,2010年度,2015年度
0,01100,人,1401757.0,1542979.0,1671742.0,1757025.0,1822368.0,1880863.0,1913545.0,1952356.0
1,01101,人,181806.0,180845.0,179184.0,173358.0,181383.0,202801.0,220189.0,237627.0
2,01102,人,195370.0,212508.0,230918.0,251419.0,260114.0,272877.0,278781.0,285321.0
3,01103,人,213310.0,224539.0,232999.0,241319.0,248950.0,253996.0,255873.0,261912.0
4,01104,人,228061.0,263938.0,188043.0,192102.0,197223.0,201307.0,204259.0,209584.0
...,...,...,...,...,...,...,...,...,...,...
1912,47361,人,10187.0,10238.0,10309.0,9819.0,9359.0,9177.0,8519.0,7755.0
1913,47362,人,18990.0,19918.0,20718.0,23033.0,24626.0,25121.0,26681.0,29066.0
1914,47375,人,1667.0,1632.0,1463.0,1409.0,1338.0,1370.0,1231.0,1194.0
1915,47381,人,3376.0,3467.0,3468.0,3508.0,3551.0,4192.0,3859.0,3998.0
